In [1]:
import os
import pandas as pd

rpkm_type='contig' #contig or fna

# 预定义的基因列表及其顺序
genes_list = [
    'katE', 'oxyR', 'sodA', 'sodB', 'soxR', 'ahpC', 'dps', 'SulA', 'recA', 'lexA', 'uvrA', 
    'umuC', 'umuD', 'ompA', 'ompC', 'ttgC', 'ttgB', 'ttgA', 'tolC', 'acrB', 'acrA'
]
path2filterm8="/storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation/ROS_gene_mmseq_contig_level_out/sustech_gene_out/filter_m8"
# 定义文件路径
input_directory = os.path.join(path2filterm8,"with_rpkg_"+rpkm_type)

# 创建一个空的 DataFrame，用于存储 abundance 表格
abundance_df = pd.DataFrame()

# 遍历 with_rpkg 目录中的所有文件
for filename in os.listdir(input_directory):
    if filename.endswith(".txt") or filename.endswith(".csv"):  # 假设文件是以 .txt 或 .csv 结尾
        # 获取文件的完整路径
        file_path = os.path.join(input_directory, filename)

        # 读取文件 (假设文件中已经有表头)
        m8_df = pd.read_csv(file_path, sep="\t")

        # 确保文件包含必要的列
        if 'Gene' not in m8_df.columns or 'Sample' not in m8_df.columns or 'RPKG' not in m8_df.columns:
            print(f"File {filename} is missing required columns. Skipping...")
            continue

        # 分组计算每个样本中每个基因的平均 RPKG 值
        avg_rpkg_df = m8_df.groupby(['Sample', 'Gene'])['RPKG'].mean().reset_index()

        # 将数据透视为 Sample 为行，Gene 为列，RPKG 为值的形式
        pivot_df = avg_rpkg_df.pivot(index='Sample', columns='Gene', values='RPKG')

        # 确保每个样本都包含所有基因，缺失的基因填充 0
        pivot_df = pivot_df.reindex(columns=genes_list, fill_value=0)

        # 将当前文件的内容合并到总的 abundance_df 中
        abundance_df = pd.concat([abundance_df, pivot_df], axis=0)

# 对 abundance_df 进行聚合，确保同一个 Sample 的基因值合并并且平均
abundance_df = abundance_df.groupby(abundance_df.index).mean()

# 将 NaN 值替换为 0 (保险起见)
abundance_df.fillna(0, inplace=True)

# 保存结果到 CSV 文件
output_file = os.path.join(input_directory, "abundance_table.csv")
abundance_df.to_csv(output_file)

print(f"Abundance table saved to {output_file}")

/tmp/ipykernel_3532843/2246111565.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Abundance table saved to /storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation/ROS_gene_mmseq_contig_level_out/sustech_gene_out/filter_m8/with_rpkg_contig/abundance_table.csv
